<a href="https://colab.research.google.com/github/kondapalliDeepa/FMML-Module-5-Project/blob/main/Module_5_project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import os

# Step 1: Data Collection and Preprocessing
def load_data(data_dir):
    if not os.path.exists(data_dir):
        raise FileNotFoundError(f"The dataset directory '{data_dir}' does not exist. Please provide the correct path.")

    images = []
    labels = []
    for label in os.listdir(data_dir):
        label_path = os.path.join(data_dir, label)
        if os.path.isdir(label_path):
            for img_file in os.listdir(label_path):
                img_path = os.path.join(label_path, img_file)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (64, 64))  # Resize to 64x64
                images.append(img.flatten())  # Flatten the image
                labels.append(int(label))  # Label corresponds to the number of fingers
    return np.array(images), np.array(labels)

# Load the dataset (replace 'data/fingers' with your dataset path)
data_dir = 'data/fingers'
try:
    X, y = load_data(data_dir)
except FileNotFoundError as e:
    print(e)
    # Create a dummy dataset for testing
    print("Creating a dummy dataset for testing...")
    os.makedirs(data_dir, exist_ok=True)
    for i in range(6):  # Labels 0 to 5
        os.makedirs(f"{data_dir}/{i}", exist_ok=True)
        for j in range(10):  # 10 images per label
            dummy_img = np.random.randint(0, 256, (64, 64), dtype=np.uint8)
            cv2.imwrite(f"{data_dir}/{i}/img{j}.png", dummy_img)
    X, y = load_data(data_dir)

# Normalize the data
X = X / 255.0

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Model Training
# 1. Decision Tree
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, y_train)

# 2. Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# 3. Support Vector Machine
svm_model = SVC(kernel='rbf', gamma='scale')
svm_model.fit(X_train, y_train)

# Step 3: Evaluation with example values
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
    print("Classification Report:\n", classification_report(y_test, y_pred, zero_division=0))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Example: Using dummy y_true and y_pred values for classification report
y_true = [0, 1, 2, 2, 1]  # Actual labels (example values)
y_pred = [0, 0, 2, 2, 1]  # Predicted labels (example values)

print("\nExample Classification Report using example values:")
print(classification_report(y_true, y_pred, zero_division=0))

print("\nDecision Tree Evaluation:")
evaluate_model(dt_model, X_test, y_test)

print("\nRandom Forest Evaluation:")
evaluate_model(rf_model, X_test, y_test)

print("\nSVM Evaluation:")
evaluate_model(svm_model, X_test, y_test)

# Step 4: Real-time Testing (Optional)
def real_time_test(model):
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, (64, 64)).flatten() / 255.0
        prediction = model.predict([resized])[0]
        cv2.putText(frame, f"Predicted: {prediction}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow("Finger Count", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Uncomment to test in real-time
# real_time_test(svm_model)



Example Classification Report using example values:
              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       1.00      0.50      0.67         2
           2       1.00      1.00      1.00         2

    accuracy                           0.80         5
   macro avg       0.83      0.83      0.78         5
weighted avg       0.90      0.80      0.80         5


Decision Tree Evaluation:
Accuracy: 0.08
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         2
           3       0.33      0.50      0.40         2
           4       0.00      0.00      0.00         2
           5       0.00      0.00      0.00         3

    accuracy                           0.08        12
   macro avg       0.06      0.08      0.07        12
weighted avg